# Data Reading



In [0]:
%python
df = spark.read.format("parquet").option('inferschema',True).load("abfss://bronze1@latestadls.dfs.core.windows.net/Raw_Data")
display(df)


# Data Transfomration


In [0]:
%python
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
df = df.withColumn('Model_category', split(col('Model_id'),'-')[0])
display(df)
df.withColumn('Units_Sold', col('Units_Sold').cast(StringType())); display(df)


In [0]:
%python
df = df.withColumn('RevPerUnit', col('Revenue')/col('Units_Sold'))
display(df)

In [0]:
%python
display(df.groupBy('Year','BranchName').agg(sum('Units_Sold').alias('Total_Units')).sort('Year','Total_Units',ascending=[1,0]))


Databricks visualization. Run in Databricks to view.

# Data Writing


In [0]:
df.write.format("parquet").mode("overwrite").save("abfss://silver1@latestadls.dfs.core.windows.net/carsales")

Querying Silver Data




In [0]:
%sql
Select * from parquet.`abfss://silver1@latestadls.dfs.core.windows.net/carsales`